In [ ]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa para buscas de processos no STJ a partir de nomes
#

In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os

In [57]:
# Mostra diretório atual

In [58]:
print(os.getcwd())

/home/abraji/Documentos/Code/coda2019/repo/raspatribunais_stj


In [59]:
# Acesssa lista de políticos

In [60]:
candidatos = pd.read_csv('exemplo_candidatos.csv', encoding='utf-8', converters={'cpf': lambda x: str(x)})

In [61]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
nome completo        4 non-null object
cpf                  4 non-null object
cargo_disputado      4 non-null object
eleicao_disputado    4 non-null int64
dtypes: int64(1), object(3)
memory usage: 256.0+ bytes


In [62]:
# Indica os locais do Chrome e do Chromedriver

In [63]:
# Estão são os locais de exemplo em um computador da ESPM com Windows - cada computador varia o endereço

In [64]:
# Por algum motivo que ainda não entendi o Chromedriver mostra um erro quando
# busco por nomes com mais de uma página
# CHROME = r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'
# CHROMEDRIVER = r'C:\Users\Administrador\Desktop\chromedriver_win32\chromedriver'

In [73]:
# Por isso testei com o geckodriver, do Mozilla, e aí deu certo
firefoxPath = "/home/abraji/Documentos/Code/geckodriver"

In [74]:
# Definir opções do Chrome 
# chrome_options = webdriver.chrome.options.Options()
#chrome_options.add_argument('--window-size=1920,1080')
# chrome_options.binary_location = CHROME

# Esta opção deixa o navegador fora da tela, ao fundo
# chrome_options.add_argument('--headless')

In [51]:
# Indica a url principal para buscas

In [75]:
link = 'https://ww2.stj.jus.br/processo/pesquisa/?aplicacao=processos.ea'

In [76]:
# Rotina principal

In [77]:
processos = []
#Itera entre os nomes de candidatos
for num, row in candidatos.iterrows():
    
    # Acessa o nome e dados gerais dos politicos
    nome = row['nome completo']
    cpf = row['cpf']
    print(nome)
    
    # Chama o driver e acessa a primeira página de busca
    #driver = webdriver.Chrome(CHROMEDRIVER, options = chrome_options)
    driver = webdriver.Firefox(executable_path=firefoxPath)
    driver.get(link)
    
    # Posiciona cursor, preenche e clica
    # Aqui exemplo de outro tipo de item que pode ser buscado, o CSS
    # Cascading Style Sheets (CSS) é uma "folha de estilo" composta por “camadas” e utilizada para 
    # definir a apresentação (aparência) em páginas da internet que adotam para o seu desenvolvimento linguagens de marcação (como XML, HTML e XHTML)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#idParteNome'))).click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="idParteNome"]').send_keys(nome)
    time.sleep(6)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#idBotaoPesquisarFormularioExtendido'))).click()
    
    # 1 - CAPTURA A QUANTIDADE DE GRAFIAS DE NOMES
    time.sleep(10)
    grafs = driver.find_element_by_xpath('//*[@id="idDivBlocoMensagem"]/div/b').text
    print("Quantidade de grafias: ", grafs)
    grafs = int(grafs)
    
    # 2 - SE TIVER APENAS UMA GRAFIA
    if grafs == 1:
        
        # 2.1 - Clica na página do processo
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#idBlocoPessoasInterno > div > span > a > span'))).click()
        time.sleep(1)
        
        # 2.2 - Testa se o nome tem mais de um processo
        # Parte do princípio que existe mais de um
        tipo = "mais_de_um"
        try:
            driver.find_element_by_xpath('//*[@id="idSpanMensagem"]')
        except NoSuchElementException:
            tipo = "um"
        
        print(tipo)
        
        if tipo == "um":
            # Executa testes para pegar os dados
            try:
                num_unico = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[6]/span[2]/a').text
            except NoSuchElementException:
                num_unico = "sem_numero_unico"
            
            try:
                nome_proc = driver.find_element_by_xpath('//*[@id="idSpanClasseDescricao"]').text
            except NoSuchElementException:
                nome_proc = "sem_nome_encontrado"
            
            try:
                data_autu = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[5]/span[2]').text
            except NoSuchElementException:
                data_autu = "sem_data_encontrada"
        
            # Preenche dicionário e lista
            dicionario = {"num_unico": num_unico,
                          "nome_proc": nome_proc,
                          "data_autu": data_autu,
                          "nome": nome,
                          "cpf": cpf
                             }
            processos.append(dicionario)
            
        elif tipo == "mais_de_um":
            qt = driver.find_element_by_xpath('//*[@id="idDivBlocoMensagem"]/div/b').text
            print("Quantidade de processos: ", qt)
            qt = int(qt) + 2 # Mais 2 porque a lista começa por 2
        
            # Faz uma iteração a partir do número encontrado de processos
            for item in range(2, qt):
            
                # Encontra o XPATH de cada link de processo
                vez = '//*[@id="idBlocoInternoLinhasProcesso"]/div[' + str(item) + ']/span[1]/span[1]/span[1]/span[2]/a'
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, vez))).click()
                time.sleep(1)
            
                # Executa testes para pegar os dados
                try:
                    num_unico = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[6]/span[2]/a').text
                except NoSuchElementException:
                    num_unico = "sem_numero_unico"
            
                try:
                    nome_proc = driver.find_element_by_xpath('//*[@id="idSpanClasseDescricao"]').text
                except NoSuchElementException:
                    nome_proc = "sem_nome_encontrado"
            
                try:
                    data_autu = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[5]/span[2]').text
                except NoSuchElementException:
                    data_autu = "sem_data_encontrada"
        
                # Preenche dicionário e lista
                dicionario = {"num_unico": num_unico,
                          "nome_proc": nome_proc,
                          "data_autu": data_autu,
                          "nome": nome,
                          "cpf": cpf
                             }
                processos.append(dicionario)
        
                # Retorna uma página para clicar no próximo processo
                driver.execute_script("window.history.go(-1)")
        
    # 3 - SE TIVER MAIS DE UMA GRAFIA
    elif grafs > 1:
        # 3.1 - Marca todos as possibilidades de grafia para buscar
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#idBotaoMarcarTodos'))).click()
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#idBotaoPesquisarMarcados'))).click()
        time.sleep(1)
        
        # 3.2 - Testa a paginação de processos
        tipo = "mais_de_uma_pag"
        # Parte do princípio que existe mais de uma
        try:
            driver.find_element_by_xpath('//*[@id="idDivBlocoPaginacaoTopo"]/div/span/span[2]')
        except NoSuchElementException:
            tipo = "uma_pag"
        
        print(tipo)
        
        
        if tipo == "uma_pag":
            # Captura a quantidade de processos encontrados
            qt = driver.find_element_by_xpath('//*[@id="idDivBlocoMensagem"]/div/b').text
            print("Quantidade de processos: ", qt)
            qt = int(qt) + 2 # Mais 2 porque a lista começa por 2
        
            # Faz uma iteração a partir do número encontrado de processos
            for item in range(2, qt):
            
                # Encontra o XPATH de cada link de processo
                vez = '//*[@id="idBlocoInternoLinhasProcesso"]/div[' + str(item) + ']/span[1]/span[1]/span[1]/span[2]/a'
           
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, vez))).click()
                time.sleep(1)
            
                # Executa testes para pegar os dados
                try:
                    num_unico = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[6]/span[2]/a').text
                except NoSuchElementException:
                    num_unico = "sem_numero_unico"
            
                try:
                    nome_proc = driver.find_element_by_xpath('//*[@id="idSpanClasseDescricao"]').text
                except NoSuchElementException:
                    nome_proc = "sem_nome_encontrado"
            
                try:
                    data_autu = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[5]/span[2]').text
                except NoSuchElementException:
                    data_autu = "sem_data_encontrada"
        
                # Preenche dicionário e lista
                dicionario = {"num_unico": num_unico,
                          "nome_proc": nome_proc,
                          "data_autu": data_autu,
                          "nome": nome,
                          "cpf": cpf
                             }
                processos.append(dicionario)
        
                # Retorna uma página para clicar no próximo processo
                driver.execute_script("window.history.go(-1)")
        
        # Rotina que entra em todas páginas
        elif tipo == "mais_de_uma_pag":
            # Estabelece uma variável para espera
            wait = WebDriverWait(driver, 20)

            #...    

            while True:
                # Captura quantidade dos links da página
                links = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//span[contains(@class,'classSpanNumeroRegistro')]")))
                print("Quantidade de links por página: ", len(links))
                
                # Faz iteração de acordo com quantidade encontrada
                for i in range(1, len(links) + 1):
                    # Acessa o link direto e clica em
                    wait.until(EC.element_to_be_clickable((By.XPATH, f"(//span[contains(@class,'classSpanNumeroRegistro')])[{i}]//a"))).click()

                    # Executar testes para obter dados
                    try:
                        num_unico = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[6]/span[2]/a').text
                    except NoSuchElementException:
                        num_unico = "sem_numero_unico"

                    try:
                        nome_proc = driver.find_element_by_xpath('//*[@id="idSpanClasseDescricao"]').text
                    except NoSuchElementException:
                        nome_proc = "sem_nome_encontrado"

                    try:
                        data_autu = driver.find_element_by_xpath('//*[@id="idProcessoDetalhesBloco1"]/div[5]/span[2]').text
                    except NoSuchElementException:
                        data_autu = "sem_data_encontrada"

                    # Preenche dicionário
                    dicionario = {"num_unico": num_unico,
                          "nome_proc": nome_proc,
                          "data_autu": data_autu,
                          "nome": nome,
                          "cpf": cpf
                             }
                    processos.append(dicionario)

                    # Retorne uma página para clicar no próximo processo
                    driver.execute_script("window.history.go(-1)")

                # Espera o botão de paginação ser carregado    
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "classSpanPaginacaoImagensDireita")))
    
                # Captura o símbolo do botão próxima página
                next_page = driver.find_elements_by_css_selector(".classSpanPaginacaoProximaPagina")
    
                # Se não existir sai do loop
                if len(next_page) == 0:
                    break
                # Caso contrário clica no botão
                next_page[0].click() 
                    
        
    # Fecha o driver
    driver.quit()

AECIO NEVES DA CUNHA
Quantidade de grafias:  2
uma_pag
Quantidade de processos:  8
GERALDO JOSE RODRIGUES ALCKMIN FILHO
Quantidade de grafias:  3
uma_pag
Quantidade de processos:  18
RUI COSTA DOS SANTOS
Quantidade de grafias:  1
mais_de_um
Quantidade de processos:  2
JOSÉ ROBERTO ARRUDA
Quantidade de grafias:  26
mais_de_uma_pag
Quantidade de links por página:  40
Quantidade de links por página:  40
Quantidade de links por página:  4


In [78]:
df_processos = pd.DataFrame(processos)

In [79]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
num_unico    112 non-null object
nome_proc    112 non-null object
data_autu    112 non-null object
nome         112 non-null object
cpf          112 non-null object
dtypes: object(5)
memory usage: 4.5+ KB


In [80]:
df_processos.reset_index().head()

,index,num_unico,nome_proc,data_autu,nome,cpf
0,0,2203591-93.2016.8.26.0000,AREsp nº 1380941 / SP,15/10/2018,AECIO NEVES DA CUNHA,66728983791
1,1,2529203-55.2010.8.13.0024,REsp nº 1702733 / MG,11/11/2016,AECIO NEVES DA CUNHA,66728983791
2,2,0228610-78.2016.3.00.0000,IJ nº 143 / DF,23/08/2016,AECIO NEVES DA CUNHA,66728983791
3,3,0146270-33.2014.8.07.0001,AREsp nº 835270 / DF,13/01/2016,AECIO NEVES DA CUNHA,66728983791
4,4,0140599-31.2010.8.13.0024,REsp nº 1538673 / MG,22/06/2015,AECIO NEVES DA CUNHA,66728983791


In [81]:
df_processos.to_excel('processos_politicos_stj.xlsx',sheet_name='Sheet1')